In [1]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset,set_architecture
from sklearn.model_selection import StratifiedShuffleSplit

/home/seonhye/.local/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
train_data1 = datasets.CIFAR10(
    root = 'data',
    train = True,                         
    transform = transform, 
    download = True,            
)
test_data1 = datasets.CIFAR10(
    root = 'data', 
    train = False, 
    transform = transform
)
train_data2 = datasets.CIFAR100(
    root = 'data',
    train = True,                         
    transform = transform, 
    download = True,            
)
test_data2 = datasets.CIFAR100(
    root = 'data', 
    train = False, 
    transform = transform
)


label_list=[0,8,16,70,22]
for i in range(50000):
  if train_data2.targets[i] not in label_list:
    train_data2.targets[i]=100
  if train_data2.targets[i]==0:
    train_data2.targets[i]=10
  elif train_data2.targets[i]==8:
    train_data2.targets[i]=11
  elif train_data2.targets[i]==16:
    train_data2.targets[i]=12
  elif train_data2.targets[i]==70:
    train_data2.targets[i]=13
  elif train_data2.targets[i]==22:
    train_data2.targets[i]=14

label_list=[0,8,16,70,22]
for i in range(10000):
  if test_data2.targets[i] not in label_list:
    test_data2.targets[i]=100
  if test_data2.targets[i]==0:
    test_data2.targets[i]=10
  elif test_data2.targets[i]==8:
    test_data2.targets[i]=11
  elif test_data2.targets[i]==16:
    test_data2.targets[i]=12
  elif test_data2.targets[i]==70:
    test_data2.targets[i]=13
  elif test_data2.targets[i]==22:
    test_data2.targets[i]=14

k=0
for i in range(50000):
  if train_data2.targets[i]!=100:
    if k==0:
      sub_data=train_data2.data[i]
      sub_data=sub_data.reshape(1,32,32,3)
      k=1
    else:
      sub_data=np.append(sub_data,train_data2.data[i].reshape(1,32,32,3),axis=0)

sub_target=[]
for i in range(50000):
  if train_data2.targets[i]!=100:
    sub_target.append(train_data2.targets[i])
train_data2.data=sub_data
train_data2.targets=sub_target

k=0
for i in range(10000):
  if test_data2.targets[i]!=100:
    if k==0:
      sub_data=test_data2.data[i]
      sub_data=sub_data.reshape(1,32,32,3)
      k=1
    else:
      sub_data=np.append(sub_data,test_data2.data[i].reshape(1,32,32,3),axis=0)
sub_target=[]
for i in range(10000):
  if test_data2.targets[i]!=100:
    sub_target.append(test_data2.targets[i])
test_data2.data=sub_data
test_data2.targets=sub_target

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_data = torch.utils.data.ConcatDataset([train_data1, train_data2])
test_data = torch.utils.data.ConcatDataset([test_data1, test_data2])

In [6]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_data,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_data,
                batch_size=batch_size,
                shuffle=True)

In [7]:
target_model = torch.load("./models/victims/Cifar10_Resnet18.pt",map_location='cpu')
fc = target_model.fc
in_dim = fc.in_features
target_model.fc = nn.Linear(in_dim,15)
target_model = target_model.to(device)

In [8]:
ct = 0
for child in target_model.children():
  ct += 1
  if ct < 7:
      for param in child.parameters():
          param.requires_grad = False

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
if not os.path.exists("./models/attack_model/DATLA"):
    os.makedirs("./models/attack_model/DATLA")
for epochs in range(5):
    model, acc = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 20*(epochs+1))
    torch.save(model,'./models/attack_model/DATLA/DATLA_'+str(20*(epochs+1))+'.pt')
    acc_file=open('./models/attack_model/DATLA/DATLA_'+str(20*(epochs+1))+'.txt','w')
    acc_file.write(str(acc))
    acc_file.close()

Training batch 400/411
Validation batch 0/83

/data/seonhye/sorce/DeepTaster/utils/train.py:195: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.to(device), volatile=True), Variable(labels.to(device), volatile=True)



Epoch 0 result: 
Avg loss (train): 0.0045
Avg acc (train): 0.8277
Avg loss (val): 0.0085
Avg acc (val): 0.7042
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 5 result: 
Avg loss (train): 0.0015
Avg acc (train): 0.9374
Avg loss (val): 0.0099
Avg acc (val): 0.7094
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 10 result: 
Avg loss (train): 0.0013
Avg acc (train): 0.9444
Avg loss (val): 0.0103
Avg acc (val): 0.7075
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 15 result: 
Avg loss (train): 0.0012
Avg acc (train): 0.9499
Avg loss (val): 0.0105
Avg acc (val): 0.7039
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Va

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 35 result: 
Avg loss (train): 0.0011
Avg acc (train): 0.9543
Avg loss (val): 0.0104
Avg acc (val): 0.7088
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 40 result: 
Avg loss (train): 0.0010
Avg acc (train): 0.9548
Avg loss (val): 0.0103
Avg acc (val): 0.7077
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 45 result: 
Avg loss (train): 0.0011
Avg acc (train): 0.9521
Avg loss (val): 0.0105
Avg acc (val): 0.7092
----------

Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Training batch 400/411
Validation batch 0/83
Epoch 50 result: 
Avg loss (train): 0.0011
Avg acc (train): 0.9549
Avg loss (val): 0.0114
Avg acc (val)